In [2]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import LocalOutlierFactor
from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np


In [32]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [1]:
def test(alpha=34,**kwargs):
    print(kwargs)

In [4]:
test(alphas=3,ff=3)

{'alphas': 3, 'ff': 3}


In [34]:
x = iris.drop(columns=['species'])
y = iris['species']

In [63]:
class DFSimpleImputer(TransformerMixin):
    def fit_transform(self,x,y):
        print('fit/trans')
        self.fit(x,y)
        return self.transform(x)
    def transform(self, x):
        print('trans')
        print(x.shape)
        return x
    def fit(self,x,y):
        print('fit')
        return x

In [64]:
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline(steps=[
    ('impute_num', DFSimpleImputer()),
    ('model',RandomForestClassifier())
    ])

In [ ]:
from sklearn import set_config
set_config(display='diagram')   
# diplays HTML representation in a jupyter context
column_trans  

In [65]:
pipe.fit(x,y)

fit/trans
fit
trans
(150, 4)


Pipeline(memory=None,
         steps=[('impute_num',
                 <__main__.DFSimpleImputer object at 0x000001E3D4769860>),
                ('model',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [66]:
from sklearn.model_selection import cross_validate
cross_validate(pipe, x,y,cv=3)

fit/trans
fit
trans
(100, 4)
trans
(50, 4)
fit/trans
fit
trans
(100, 4)
trans
(50, 4)
fit/trans
fit
trans
(100, 4)
trans
(50, 4)


{'fit_time': array([0.46672773, 0.59866142, 0.60865641]),
 'score_time': array([0.04297948, 0.03597403, 0.03497887]),
 'test_score': array([0.98, 0.94, 0.98])}

In [42]:
pipe.fit(x,y)

fit/trans


Pipeline(memory=None,
         steps=[('impute_num',
                 <__main__.DFSimpleImputer object at 0x000001E3D36A1D68>),
                ('model',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=100, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [1]:
import pandas as pd

In [19]:
import numpy as np
df= pd.DataFrame({'col1':[1,2,3,4],'col2':[0,0,0,100]})
df

,col1,col2
0,1,0
1,2,0
2,3,0
3,4,100


In [4]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.impute import SimpleImputer
class DFZeroOrMeanImputer(TransformerMixin): # only for numeric features
    def fit(self, X, y=None):
        def determine_zero(series):
            perc_zero =  sum(series==0) / series.count()
            if perc_zero > 0.05: #zero occurs often: 
                return True
            else: 
                return False
        
        self.zero_cols = set([c for c in X.columns if determine_zero(X[c])])
        self.other_cols = set(X.columns) - self.zero_cols
        
        if self.zero_cols:
            self.zero_imp = SimpleImputer(strategy='constant', fill_value=0)
            self.zero_imp.fit(X[self.zero_cols])

        if self.other_cols:
            self.other_imp = SimpleImputer(strategy='mean')
            self.other_imp.fit(X[self.other_cols])
        return self
    
    def transform(self, X,y=None):
        zero_trans, other_trans = pd.DataFrame(),pd.DataFrame(),
        if self.zero_cols: zero_trans = pd.DataFrame(self.zero_imp.transform(X[self.zero_cols]), columns = self.zero_cols)
        if self.other_cols: other_trans = pd.DataFrame(self.other_imp.transform(X[self.other_cols]), columns = self.other_cols)
        return pd.concat([other_trans,zero_trans], axis=1)

In [32]:
from sklearn.neighbors import LocalOutlierFactor
class DFOutlierExtractor(TransformerMixin):

    def __init__(self, model, thres=-1.5, contamination=None):
        """ 
        Keyword Args:
        neg_conf_val (float): The threshold for excluding samples with a lower
        negative outlier factor.
        """
        self.model = model
        self.threshold = thres
        self.contamination = contamination
    def __repr__(self):
        return f'DFOutlierExtractor with threshold {self.threshold} and model {self.model}'

    def fit(self, X, y):
        xs = np.asarray(X)
        ys = np.asarray(y)
        if self.contamination: 
            lcf = LocalOutlierFactor(contamination=self.contamination)
        else:
            lcf = LocalOutlierFactor()
        lcf = lcf.fit(xs)
        print(lcf.offset_)
        if self.contamination: self.threshold = lcf.offset_
        xs  = pd.DataFrame(xs[lcf.negative_outlier_factor_ > self.threshold, :],columns=X.columns)
        ys = y[lcf.negative_outlier_factor_ > self.threshold]
        print('removed',len(X) - len(xs),self.threshold, 'thres',self.threshold)
        self.model.fit(xs,ys)
        return self

    def predict(self, X):
        return self.model.predict(X)


In [43]:
from sklearn.linear_model import Lasso
out = DFOutlierExtractor(Lasso(),contamination=0.42)

In [44]:
out.fit(df,df['col2'])

-1.0000099948728338
removed 2 -1.0000099948728338


DFOutlierExtractor with threshold -1.0000099948728338 and model Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [31]:
[imp.overall_imp[i] if pd.isna(imp_val) else imp_val for i, imp_val in enumerate(imp.specific_imps['MeadowV'].statistics_)]

NameError: name 'imp' is not defined

In [276]:
imp.overall_imp

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [264]:
imp.specific_imps['MeadowV'].statistics_[70] == imp.specific_imps['MeadowV'].statistics_[70]

False

In [257]:
import math 
math.isnan(imp.specific_imps['MeadowV'].statistics_[70])

True

In [265]:
pd.isna(imp.specific_imps['MeadowV'].statistics_[70])

True

In [117]:
si = SimpleImputer(add_indicator=True)
si.fit_transform(df)

array([[  1. ,   0. ,   0. ,   0. ],
       [  2. ,   0. ,   0. ,   0. ],
       [  3. ,   0. ,   0. ,   0. ],
       [  4. , 100. ,   0. ,   0. ],
       [  2.5,  25. ,   1. ,   1. ],
       [  2.5,  25. ,   1. ,   1. ]])

In [119]:
b = 'a'
f'2{b}'

'2a'

In [279]:
aoc.nan_inspect(a)

0    1456
dtype: int64


ValueError: cannot concatenate unaligned mixed dimensional NDFrame objects

In [292]:
train_x.drop(columns=['Neighborhood']).shape

(1456, 176)

In [298]:
from sklearn.compose import make_column_selector
m = make_column_selector(dtype_exclude=np.number)

In [334]:
class test():
    def __init__(self,dtype_exclude):
        self.dtype_exclude = dtype_exclude
    def __call__(self,train):
        return make_column_selector(dtype_exclude = self.dtype_exclude)(train)+['Neighborhood']

In [335]:
from sklearn.compose import make_column_selector
a = test(dtype_exclude=np.number)

In [336]:
b = a(train_x)

In [98]:
def bla(**kwargs):
    print(kwargs)
bla(hi=2,isl=3)

{'hi': 2, 'isl': 3}


In [109]:
df

,col1,col2
0,1.0,0.0
1,2.0,0.0
2,3.0,0.0
3,4.0,100.0
4,NaN,NaN
5,NaN,NaN


In [111]:
cols = df.columns.to_list()
random.shuffle(cols,random_state=34)
df[cols]

TypeError: shuffle() got an unexpected keyword argument 'random_state'

In [107]:
cols